توضیحات بیشتر در فایل پی دی اف توضیحات

In [33]:
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt

لود شدن تصویر و تشخیص مرز ها و حذف سطر های بدون نقاط مرزی

In [34]:
img = cv2.imread('birds.jpg', cv2.IMREAD_UNCHANGED)
edges = cv2.Canny(img,100,200)

idx = np.argwhere(np.all(edges[..., :] == 0, axis=1))
edg_croped = np.delete(edges, idx, axis=0)
img_croped = np.zeros((edg_croped.shape[0],edg_croped.shape[1],3), dtype = 'uint8')
img_croped[:,:,0] = np.delete(img[:,:,0] , idx, axis=0)
img_croped[:,:,1] = np.delete(img[:,:,1] , idx, axis=0)
img_croped[:,:,2] = np.delete(img[:,:,2] , idx, axis=0)


لود شدن سمپل کبوتر در فضای خاکستری

In [35]:

img_gray = cv2.cvtColor(img_croped, cv2.COLOR_BGR2GRAY)


لود شدن تمپلیت های نمونه

In [36]:
template = cv2.imread('sample1.jpg',cv2.IMREAD_GRAYSCALE)
# template1 = cv2.imread('sample2.jpg',cv2.IMREAD_GRAYSCALE)
template2 = cv2.imread('sample3.jpg',cv2.IMREAD_GRAYSCALE)

width = template.shape[1]
height = template.shape[0]

تمپلیت مچینگ

In [37]:
matched_interval = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
threshold = 0.6038
args = np.where(matched_interval >= threshold)
# for pt in zip(*args[::-1]):
#     cv2.rectangle(img_croped, pt, (pt[0] + width, pt[1] + height), (0,0,255), 2)

# matched_interval = cv2.matchTemplate(img_gray,template1,cv2.TM_CCOEFF_NORMED)
# threshold = 0.8
# args = np.where(matched_interval >= threshold)
# for pt in zip(*args[::-1]):
#     cv2.rectangle(img_croped, pt, (pt[0] + width, pt[1] + height), (0,0,255), 2)

# cv2.imwrite('res.png',img_croped)

xargs = args[0]
yargs = args[1]


اجرای گرب کات به ازای تعدادی از مستطیل های کوچک که در آن کبوتری وجود دارد

In [38]:
# cv.imwrite('res.png',output)

mask = np.zeros(img.shape[:2], np.uint8)

bgdModel = np.zeros((1, 65), np.float64)
fgdModel = np.zeros((1, 65), np.float64)
print('before')
# print(img_croped.shape)
mask = np.zeros((img_croped.shape[0],img_croped.shape[1])).astype('bool')
for i in range(0,xargs.shape[0]):
    if i < 5  or abs(xargs[i] - xargs[i-1]) > 2 or abs(yargs[i] - yargs[i-1]) > 2 or i%4==1 or i>xargs.shape[0]-5:
        rect = (1,1,  width-1,  height-1)
        # print(rect)
        imi = np.zeros((width,height,3),dtype = np.uint8)
        masky = np.zeros(imi.shape[:2],np.uint8)

        # print(img_croped.shape)
        imi[:,:,:] = img_croped[ xargs[i]: xargs[i]+width ,yargs[i]:yargs[i] + height,:]
        # print(imi)
        cv2.grabCut(imi,masky,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
        # print('-------')
        mask2 = np.where((masky == 2) | (masky == 0), 0, 1)
        # print(mask2.shape)
        # print(mask.shape)
        mask[ xargs[i]: xargs[i]+width ,yargs[i]:yargs[i] + height] = mask[ xargs[i]: xargs[i]+width ,yargs[i]:yargs[i] + height] | mask2
        #a little edition
        mask[xargs[i]+width:xargs[i]+width+20,yargs[i]+7:yargs[i] + height-30] = True
nahai1 = mask
# imaged = img_croped * nahai[:, :, np.newaxis]
# print('after1')

# for pt in zip(*args[::-1]):
#     cv2.rectangle(img_croped, pt, (pt[0] + width, pt[1] + height), (0,0,255), 2)


# cv.imwrite('res.png',output)



before


تکرار اعمال بالا با یک سمپل دیگر برای دقیق تر شدن کار

In [39]:

matched_interval = cv2.matchTemplate(img_gray,template2,cv2.TM_CCOEFF_NORMED)
threshold = 0.67
args = np.where(matched_interval >= threshold)

In [40]:
xargs = args[0]
yargs = args[1]


mask = np.zeros(img.shape[:2], np.uint8)

bgdModel = np.zeros((1, 65), np.float64)
fgdModel = np.zeros((1, 65), np.float64)
# print('before2')
# print(img_croped.shape)
mask = np.zeros((img_croped.shape[0],img_croped.shape[1])).astype('bool')
for i in range(0,xargs.shape[0]):
    if i < 5  or abs(xargs[i] - xargs[i-1]) > 2 or abs(yargs[i] - yargs[i-1]) > 2 or i%4==1 or i>xargs.shape[0]-5:
        rect = (1,1,  width-1,  height-1)
        # print(rect)
        imi = np.zeros((width,height,3),dtype = np.uint8)
        masky = np.zeros(imi.shape[:2],np.uint8)

        # print(img_croped.shape)
        imi[:,:,:] = img_croped[ xargs[i]: xargs[i]+width ,yargs[i]:yargs[i] + height,:]
        # print(imi)
        cv2.grabCut(imi,masky,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
        # print('-------')
        mask2 = np.where((masky == 2) | (masky == 0), 0, 1)
        # print(mask2.shape)
        # print(mask.shape)
        mask[ xargs[i]: xargs[i]+width ,yargs[i]:yargs[i] + height] = mask[ xargs[i]: xargs[i]+width ,yargs[i]:yargs[i] + height] | mask2
        #a little edition
        mask[xargs[i]+width:xargs[i]+width+20,yargs[i]+7:yargs[i] + height-30] = True
nahai = mask | nahai1
nahai = nahai.astype(int)
imaged = img_croped * nahai[:, :, np.newaxis]
# print('after2')

# plt.imshow(imaged),plt.colorbar(),plt.show()
cv2.imwrite('res.jpg', imaged)
# files.download('res08.jpg')


after2


True